# [STARTER] Exercise - Building an Agentic RAG System

In this exercise, you will build an Agentic RAG (Retrieval-Augmented Generation) system that 
combines the power of AI agents with traditional RAG pipelines. You'll create an agent that 
can decide when and how to retrieve information from different sources, including vector 
databases, web search, and other tools.


## Challenge

Your challenge is to create an Agentic RAG system that can:

- Build a RAG pipeline as a tool that can be used by the agent
- Create an agent that can decide which tool to use based on the query
- Handle different types of queries intelligently
- Combine information from multiple sources when needed


## Setup
First, let's import the necessary libraries:

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
import os
from typing import List
from dotenv import load_dotenv

from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager, CorpusLoaderService
from lib.rag import RAG

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Load data to Vector DB

In [3]:
db = VectorStoreManager(OPENAI_API_KEY)
db

VectorStoreManager():<chromadb.api.client.Client object at 0x120206090>

In [4]:
loader_service = CorpusLoaderService(db)

In [5]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [6]:
# TODO: Add the games pdf file path with the extenstion .pdf
# And define a store name in load_pdf() method

file_path = "TheGamingIndustry2024.pdf"



games_market_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf(store_name="gaming_industry", pdf_path=file_path)
)

VectorStore `gaming_industry` ready!
Pages from `TheGamingIndustry2024.pdf` added!


In [7]:
result:Run = games_market_rag.invoke(
    "What's the  state of virtual reality"
)
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The state of virtual reality (VR) in 2024 is characterized by its significant impact on the gaming industry, allowing players to immerse themselves in realistic and interactive digital environments. VR technology is empowering creators to develop engaging gaming experiences, as exemplified by games like “Half-Life: Alyx,” which features high-fidelity graphics, intuitive controls, and immersive audio. The technology is also being leveraged in educational settings, enhancing learning through immersive experiences. Overall, VR is seen as a transformative force in gaming, driving innovation and expanding the boundaries of player engagement.


In [8]:
# TODO: Add the electric vehicles pdf file path with the extenstion .pdf
# And define a store name in load_pdf() method

ev_file_path = "GlobalEVOutlook2025.pdf"

electric_vehicles_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf(store_name="ev_outlook",pdf_path=ev_file_path)
)

VectorStore `ev_outlook` ready!
Pages from `GlobalEVOutlook2025.pdf` added!


In [9]:
result:Run = electric_vehicles_rag.invoke("What was the number of electric car sales and their market share in Brazil in 2024?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
In 2024, Brazil had nearly 125,000 electric car sales, which represented a market share of 6.5%.


## Tools

In a simple form, Agentic RAG can act like a router, choosing between multiple external sources to retrieve relevant information. These sources aren't limited to databases, they can also include tools like web search or APIs for services such as Slack or email.

In this case it will choose between two collections.

In [10]:
# TODO: Define a tool that returns result.get_final_state()["answer"]
# DONOT Forget about defining the tool docstrings
@tool
def search_global_ev_collection(query):
    result:Run = games_market_rag.invoke(query)
    return result.get_final_state()["answer"]

In [11]:
# TODO: Define a tool that returns result.get_final_state()["answer"]
# DONOT Forget about defining the tool docstrings
@tool
def search_games_market_report_collection(query):
    result:Run = games_market_rag.invoke(query)
    return result.get_final_state()["answer"]

In [12]:
# TODO: Add the tools you have defined and the instructions to your agent
agentic_rag = Agent(
    model_name="gpt-4o-mini",
    tools=[search_games_market_report_collection, search_global_ev_collection],    
    instructions="You are an expert market analyst."
)

In [13]:
def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run

In [14]:
run_1 = agentic_rag.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
    session_id="oscar",
)

print("\nMessages from run 1:")
messages = run_1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an expert market analyst., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have access to real-time information or events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check a reliable news source or the official Oscars website for the most up-to-date information., tool_calls = None)


In [15]:
run_2 = agentic_rag.invoke(
    query= (
        "Which two countries accounted for most of the electric car exports from " 
        "the Asia Pacific region (excluding China) in 2024?"
    ),
    session_id="electric_car",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an expert market analyst., tool_calls = None)
 -> (role = user, content = Which two countries accounted for most of the electric car exports from the Asia Pacific region (excluding China) in 2024?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_O6ZpHaUyEspukAjsoTKf1Arh', function=Function(arguments='{"query":"electric car exports Asia Pacific 2024"}', name='search_global_ev_collection'), type='function')])
 -

In [16]:
run_3 = agentic_rag.invoke(
    query= (
        "Why is generative AI seen more as an accelerator than a replacement in game development?"
    ),
    session_id="games",
)

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are an expert market analyst., tool_calls = None)
 -> (role = user, content = Why is generative AI seen more as an accelerator than a replacement in game development?, tool_calls = None)
 -> (role = assistant, content = Generative AI is viewed more as an accelerator than a replacement in game development for several key reasons:

1. **Enhancement of Creativity**: Generative AI tools assist developers by providing inspiration and new ideas, enhancing the creative process rather than replacing human creativity. They can generate unique assets, storylines, or game mechanics, allowing developers to explore a wider range of possibilities.

2. **Efficiency in Production**: AI can automate repetitive tasks such as asset creation, level design, and testing, which speeds up 